In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_person = seznam_souboru(slozka)
seznam_person[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

In [5]:
import pandas as pd

In [6]:
def id_filmu(url):
    if "cs/film/" in url:
        id_filmu = url.split("/")[-2]
    else:
        id_filmu = 0
    return(id_filmu)

In [7]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [8]:
def scrape_role(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
        
    item_details = soup.find_all(class_='db-film-credits-section')
    
    snimek = []
    
    for item_detail in item_details:
        
        if "Hrají" in item_detail.find("h2").text:
            
            divs = item_detail.find_all(class_="credit-item")

            for d in divs:
                
                role = {}
                
                role["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""
    
                uloha = None
                hraje = None
                mluvi = None
                
                if d.find("h3").find("a"):
                    hraje = id_persony(d.find("h3").find("a")["href"])
    
                else:
                    hraje = d.find("h3").text.strip()
                    
                if d.find("br"):
                    try:
                        uloha = d.find("p").find("br").next_sibling.text.strip()
                    except:
                        pass
                    
                    if d.find("span").text.strip() == "Mluví":
                        if d.find("p").find("a"):
                            mluvi = id_persony(d.find("p").find("a")["href"])                        
                            
                else:
                    if d.find("p"):
                        uloha = d.find("p").text.strip()
                
                if uloha:
                    if "/" in uloha and uloha[-1] != "/":
                            uloha = uloha.split("/")
                            for u in uloha:
                                role["Role"] = u.strip()
                                role["Hraje"] = hraje
                                role["Mluví"] = mluvi
                    
                role["Role"] = uloha
                role["Hraje"] = hraje
                role["Mluví"] = mluvi

                snimek.append(role)
                
    return pd.DataFrame(snimek)

## Test

In [9]:
test = scrape_role("filmy","10173-eastern-krvavy-hugo.html")
test

,Film,Role,Hraje,Mluví
0,Eastern – Krvavý Hugo (10173),Krvavý Hugo,Štefan Capko,None
1,Eastern – Krvavý Hugo (10173),stařenka,Hedvika Krupičková,None
2,Eastern – Krvavý Hugo (10173),holčička,Lucie Štepánková,None
3,Eastern – Krvavý Hugo (10173),pradlenka,Denisa Nová,None
4,Eastern – Krvavý Hugo (10173),muž,3277,None
5,Eastern – Krvavý Hugo (10173),žena,Zuzana Veselá,None
6,Eastern – Krvavý Hugo (10173),None,David Heger,None
7,Eastern – Krvavý Hugo (10173),None,3004,None
8,Eastern – Krvavý Hugo (10173),komando,1189,None


In [10]:
test = scrape_role("filmy","402178-pisen-nemilovaneho.html")
test

,Film,Role,Hraje,Mluví
0,Píseň nemilovaného (402178),"básník Guillaume Apollinaire, vlastním jménem ...",111618,1674
1,Píseň nemilovaného (402178),"malířka Marie Laurencinová, přítelkyně Guillauma",Christine Boisson /ž/,1427
2,Píseň nemilovaného (402178),"učitelka Madeleine Pagèsová, snoubenka Guillauma",Béatrice Bruno /ž/,844
3,Píseň nemilovaného (402178),"Madame Angelica de Kostrowitzky, matka Guillauma",5950,None
4,Píseň nemilovaného (402178),"důstojník Francesco Flugi d’Aspermont, otčím G...",7967,None
...,...,...,...,...
113,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Luboš Hadač,None
114,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Jan Šmíd,None
115,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Bohumil Holeček,None
116,Píseň nemilovaného (402178),"[hudebník v karnevalovém průvodu, hudebník v k...",Josef Pospíšil,None


## Scrapujeme herectvo

In [11]:
def scrape_herectvo(slozka):
    herectvo = pd.DataFrame()
    count = 0 
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"done: {count} / total: {len(seznam_souboru(slozka)[:kolik])} / {s}")
        h = scrape_role(slozka, s)
        herectvo = pd.concat([herectvo, h])
    herectvo = herectvo.reset_index(drop=True)
    return(herectvo)

In [12]:
kolik = None

In [13]:
herectvo = scrape_herectvo("filmy")

done: 1 / total: 4580 / 10144-tabor-padlych-zen.html
done: 2 / total: 4580 / 10173-eastern-krvavy-hugo.html
done: 3 / total: 4580 / 102282-o-jecminkovi.html
done: 4 / total: 4580 / 102314-zimni-vila.html
done: 5 / total: 4580 / 10242-narkoblues.html
done: 6 / total: 4580 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
done: 7 / total: 4580 / 103354-obcansky-prukaz.html
done: 8 / total: 4580 / 103358-7-dni-hrichu.html
done: 9 / total: 4580 / 103360-vojta-lavicka-nahoru-a-dolu.html
done: 10 / total: 4580 / 103369-kuky-se-vraci.html
done: 11 / total: 4580 / 103372-habermannuv-mlyn.html
done: 12 / total: 4580 / 103374-dobre-placena-prochazka.html
done: 13 / total: 4580 / 103402-nebe-peklo.html
done: 14 / total: 4580 / 103495-lidice.html
done: 15 / total: 4580 / 103499-medvedi-ostrovy.html
done: 16 / total: 4580 / 103509-dont-stop.html
done: 17 / total: 4580 / 104213-az-na-veky.html
done: 18 / total: 4580 / 10574-elfilm.html
done: 19 / total: 4580 / 106663-dum.html
done: 20 / total: 4580 / 1066

done: 158 / total: 4580 / 16423-waterloo-po-cesku.html
done: 159 / total: 4580 / 164346-certova-nevesta.html
done: 160 / total: 4580 / 16439-zacatek-sveta.html
done: 161 / total: 4580 / 16450-vrazda-septanym-slovem-na-velikonoce-o-velkem-patku.html
done: 162 / total: 4580 / 16453-zatmeni-nadeji-o-ctyci-mesice-pozdeji-11-8-1999.html
done: 163 / total: 4580 / 16454-panny-posetile-o-dalsi-ctyri-mesice-pozdeji-31-12-1999.html
done: 164 / total: 4580 / 164669-westernstory.html
done: 165 / total: 4580 / 165457-praho-ma-lasko.html
done: 166 / total: 4580 / 165508-bastardi.html
done: 167 / total: 4580 / 1659-mach-sebestova-a-kouzelne-sluchatko.html
done: 168 / total: 4580 / 1661-kralovsky-slib.html
done: 169 / total: 4580 / 166198-vendeta.html
done: 170 / total: 4580 / 166202-cigan.html
done: 171 / total: 4580 / 166279-vera-68.html
done: 172 / total: 4580 / 166281-nejvetsi-prani.html
done: 173 / total: 4580 / 167017-lezaky-42.html
done: 174 / total: 4580 / 168707-rok-konopi.html
done: 175 / to

done: 311 / total: 4580 / 22394-eliska-ma-rada-divocinu.html
done: 312 / total: 4580 / 22433-ene-bene.html
done: 313 / total: 4580 / 22534-fontana-pro-zuzanu-3.html
done: 314 / total: 4580 / 22572-hanele.html
done: 315 / total: 4580 / 22875-non-plus-ultras.html
done: 316 / total: 4580 / 22888-horem-padem.html
done: 317 / total: 4580 / 22896-kamenak-2.html
done: 318 / total: 4580 / 22899-zralok-v-hlave.html
done: 319 / total: 4580 / 23443-cesky-sen.html
done: 320 / total: 4580 / 23638-choking-hazard.html
done: 321 / total: 4580 / 23639-vratne-lahve.html
done: 322 / total: 4580 / 23661-byl-jsem-mladistvym-intelektualem.html
done: 323 / total: 4580 / 23667-kanarek.html
done: 324 / total: 4580 / 23974-spravce-statku.html
done: 325 / total: 4580 / 23981-hrubes-a-mares-jsou-kamaradi-do-deste.html
done: 326 / total: 4580 / 24015-mir-jejich-dusi.html
done: 327 / total: 4580 / 24064-bolero.html
done: 328 / total: 4580 / 24194-lazebnik-sibirsky.html
done: 329 / total: 4580 / 24228-maj-kompjutr.h

done: 468 / total: 4580 / 395164-ferenc-se-zeni.html
done: 469 / total: 4580 / 395165-kozlonoh.html
done: 470 / total: 4580 / 395166-laska-si-neda-poroucet.html
done: 471 / total: 4580 / 395167-lasko-trikrat-svata.html
done: 472 / total: 4580 / 395168-na-pomoc-dohode.html
done: 473 / total: 4580 / 395169-o-devcicu.html
done: 474 / total: 4580 / 395170-osalena-komtesa-zuzana.html
done: 475 / total: 4580 / 395171-praha-roku-1549.html
done: 476 / total: 4580 / 395172-princezna-z-chalupy.html
done: 477 / total: 4580 / 395173-sen-fratera-ondreje.html
done: 478 / total: 4580 / 395174-sestnactileta.html
done: 479 / total: 4580 / 395175-ucitel-orientalnich-jazyku.html
done: 480 / total: 4580 / 395176-v-mesici-lasky.html
done: 481 / total: 4580 / 395177-ada-se-uci-jezdit.html
done: 482 / total: 4580 / 395178-akcie-p-z-h.html
done: 483 / total: 4580 / 395179-akord-smrti.html
done: 484 / total: 4580 / 395180-aloisuv-los.html
done: 485 / total: 4580 / 395181-boby-nesmi-kourit.html
done: 486 / tota

done: 623 / total: 4580 / 395320-muz-bez-srdce.html
done: 624 / total: 4580 / 395321-palicova-dcera.html
done: 625 / total: 4580 / 395322-pepanek-nezdara.html
done: 626 / total: 4580 / 395323-pro-a-proti.html
done: 627 / total: 4580 / 395324-problematicky-gentleman.html
done: 628 / total: 4580 / 395325-rasputin.html
done: 629 / total: 4580 / 395326-tu-ten-kamen.html
done: 630 / total: 4580 / 395327-unos-bankere-fuxe.html
done: 631 / total: 4580 / 395328-za-oponou-smrti.html
done: 632 / total: 4580 / 395329-zahadny-pripad-galginuv.html
done: 633 / total: 4580 / 395330-zavet-podivinova.html
done: 634 / total: 4580 / 395331-bily-raj.html
done: 635 / total: 4580 / 395332-chytte-ho.html
done: 636 / total: 4580 / 395333-dama-z-baru.html
done: 637 / total: 4580 / 395334-devce-z-hor.html
done: 638 / total: 4580 / 395335-dvoji-zivot.html
done: 639 / total: 4580 / 395336-hrichy-v-manzelstvi.html
done: 640 / total: 4580 / 395337-jindra-hrabenka-ostrovinova.html
done: 641 / total: 4580 / 395338-ku

...

done: 3890 / total: 4580 / 401839-mimi-liza.html
done: 3891 / total: 4580 / 401840-kdyz-draka-boli-hlava.html
done: 3892 / total: 4580 / 401841-zlaty-podraz.html
done: 3893 / total: 4580 / 401842-ten-kdo-te-miloval.html
done: 3894 / total: 4580 / 401843-sklep.html
done: 3895 / total: 4580 / 401844-neklidna-hranice.html
done: 3896 / total: 4580 / 401845-po-cem-muzi-touzi.html
done: 3897 / total: 4580 / 401846-cesta-vede-do-tibetu.html
done: 3898 / total: 4580 / 401847-na-stojaka-v-kine.html
done: 3899 / total: 4580 / 401848-manzelske-etudy-nova-generace.html
done: 3900 / total: 4580 / 401851-iluze.html
done: 3901 / total: 4580 / 401852-univerzity-a-svoboda.html
done: 3902 / total: 4580 / 401853-zeny-v-behu.html
done: 3903 / total: 4580 / 401854-moje-stoleti.html
done: 3904 / total: 4580 / 401855-kapela.html
done: 3905 / total: 4580 / 401856-central-bus-station.html
done: 3906 / total: 4580 / 401857-cukr.html
done: 3907 / total: 4580 / 401858-narusitel.html
done: 3908 / total: 4580 / 401

done: 4035 / total: 4580 / 401988-konzultace-posledni-den-zkouskoveho-obdobi.html
done: 4036 / total: 4580 / 401989-spolu-sami.html
done: 4037 / total: 4580 / 401990-starici.html
done: 4038 / total: 4580 / 401991-prvni-akcni-hrdina-a-jako-bonus-prvni-akcni-dokument.html
done: 4039 / total: 4580 / 401992-jeste-nekoncime.html
done: 4040 / total: 4580 / 401993-narodni-trida.html
done: 4041 / total: 4580 / 401994-pan-ponrepo-se-klani.html
done: 4042 / total: 4580 / 401995-plac-svateho-sebestiana.html
done: 4043 / total: 4580 / 401996-abstinent.html
done: 4044 / total: 4580 / 401997-prvni-akcni-hrdina.html
done: 4045 / total: 4580 / 401998-prvni-akcni-dokument.html
done: 4046 / total: 4580 / 401999-v-zajeti-site.html
done: 4047 / total: 4580 / 402000-hra.html
done: 4048 / total: 4580 / 402001-posledni-aristokratka.html
done: 4049 / total: 4580 / 402002-tiche-doteky.html
done: 4050 / total: 4580 / 402003-prazske-orgie.html
done: 4051 / total: 4580 / 402004-uzly-a-pomerance.html
done: 4052 / 

done: 4184 / total: 4580 / 402147-gump-pes-ktery-naucil-lidi-zit.html
done: 4185 / total: 4580 / 402148-muj-otec-antonin-kratochvil.html
done: 4186 / total: 4580 / 402149-na-krev.html
done: 4187 / total: 4580 / 402150-tichy-spolecnik.html
done: 4188 / total: 4580 / 402151-budiz-voda.html
done: 4189 / total: 4580 / 402152-jedine-tereza.html
done: 4190 / total: 4580 / 402153-martanske-lode.html
done: 4191 / total: 4580 / 402154-atlas-ptaku.html
done: 4192 / total: 4580 / 402155-krystof.html
done: 4193 / total: 4580 / 402156-kurz-manzelske-touhy.html
done: 4194 / total: 4580 / 402157-okupace.html
done: 4195 / total: 4580 / 402158-shoky-morthy-posledni-velka-akce.html
done: 4196 / total: 4580 / 402159-spici-mesto.html
done: 4197 / total: 4580 / 402160-jak-jsem-se-stala-partyzankou.html
done: 4198 / total: 4580 / 402162-dovolena.html
done: 4199 / total: 4580 / 402163-postacka-pohadka.html
done: 4200 / total: 4580 / 402164-rene-vezen-svobody.html
done: 4201 / total: 4580 / 402165-sila.html
d

done: 4336 / total: 4580 / 51759-tobruk.html
done: 4337 / total: 4580 / 51760-medvidek.html
done: 4338 / total: 4580 / 51777-bobule.html
done: 4339 / total: 4580 / 52494-protektor.html
done: 4340 / total: 4580 / 52699-tajnosti.html
done: 4341 / total: 4580 / 52734-ivetka-a-hora.html
done: 4342 / total: 4580 / 52854-edith-piaf.html
done: 4343 / total: 4580 / 52996-praha-ocima.html
done: 4344 / total: 4580 / 53009-karty-jsou-rozdany.html
done: 4345 / total: 4580 / 53011-absolutni-laska.html
done: 4346 / total: 4580 / 53015-obrazky-z-vyletu.html
done: 4347 / total: 4580 / 53021-riziko.html
done: 4348 / total: 4580 / 53045-sest-statecnych.html
done: 4349 / total: 4580 / 53047-prvy-statocny-vlado-kral-ked-nie-tak-nie.html
done: 4350 / total: 4580 / 53051-druhy-statocny-vojtech-masek-tomik.html
done: 4351 / total: 4580 / 53055-treti-statocny-jaro-vojtek-rojkoviasnilci.html
done: 4352 / total: 4580 / 53057-stvrty-statocny-tereza-kucerova-prazske-nokturno-aneb-komora.html
done: 4353 / total: 4

done: 4487 / total: 4580 / 80193-alois-nebel.html
done: 4488 / total: 4580 / 80194-tmar-a-jeho-rod-aneb-slzave-udoli-pyramid.html
done: 4489 / total: 4580 / 80241-cinematerapie.html
done: 4490 / total: 4580 / 80243-katka.html
done: 4491 / total: 4580 / 80254-cesky-mir.html
done: 4492 / total: 4580 / 80296-gyumri.html
done: 4493 / total: 4580 / 80375-hlava-ruce-srdce.html
done: 4494 / total: 4580 / 8045-kulate-ruiny.html
done: 4495 / total: 4580 / 8046-posledni-vanoce-meho-domu.html
done: 4496 / total: 4580 / 8047-zvlastni-prilezitost.html
done: 4497 / total: 4580 / 8048-zamek.html
done: 4498 / total: 4580 / 8049-nehrbacova-romance.html
done: 4499 / total: 4580 / 80516-pouta.html
done: 4500 / total: 4580 / 80636-osmdesat-dopisu.html
done: 4501 / total: 4580 / 8066-prilis-hlucna-samota.html
done: 4502 / total: 4580 / 80794-zoufalci.html
done: 4503 / total: 4580 / 80858-trafacka-chram-svobody.html
done: 4504 / total: 4580 / 80918-oko-nad-prahou.html
done: 4505 / total: 4580 / 8141-v-erbu-

## Export

In [14]:
import os

In [15]:
herectvo.to_json(os.path.join("data","herectvo_raw.json"),orient="records")